In [1]:
import random
import sys
import os

### exp_mod es una funcion de exponenciacion modular rapida.
exp_mod(a: int, b: int, n: int) -> int

En esta implementacion, transformamos nuestro
numero a su representacion binaria.

Se traversa el numero binario de izquierda a derecha

Para este tipo de exponenciacion dividimos el exponente
en 2 partes, por ejemplo
b = 3 (0b10)
b_1 = 2, b_2 = 1
Cuando dividimos un numero impar nos quedan 2 numeros distintos.
este _extra_ significa que tendremos que multiplicar la base 1 vez mas.
este _extra_ esta representado cuando hay un 1 en la representacion binaria.

En particular siempre se parte con un _extra_ a menos que el exponente sea 0.

In [3]:
def exp_mod(a, b, n):
    if b == 0:
        return 1
    binary_b = bin(b)[2:]
    r = 1
    for i in range(len(binary_b)):
        r = (r * r) % n
        if binary_b[i] == "1":
            r = (r * a) % n
    return r

### miller_rabin es una funcion que comprueba si un numero es primo o no.

miller_rabin(n: int, k: int) -> bool

La funcion descompone el numero (n - 1) de la forma $2^t * s$. Luego elige un testigo a < n - 1 de manera aleatoria. Este testigo prueba si n era un numero primo. La probabilidad de error en este test es de 1/4.

Para lograr una probabilidad de error menor a $2^{-k}$ es necesario hacer tener k/2 + 1 testigos (repetir el test esa cantidad de veces).

In [5]:
def miller_rabin(n, k):
    s = n - 1
    t = 0
    while s % 2 == 0:
        s = s // 2
        t += 1
    for _ in range((k//2) + 1):
        a = random.randrange(2, n - 1)
        v = exp_mod(a, s, n)
        if v != 1:
            i = 0
            while v != (n - 1):
                if i >= t - 1:
                    return False
                else:
                    i = i + 1
                    v = exp_mod(v, 2, n)
    return True

### generar_primo es una funcion que genera un primo aleatorio

generar_primo(L: int) -> int

Para generar el primo aprovechamos el test de primalidad definido en miller_rabin. elegimos un numero aleatorio cercano a largo L y probamos si es un primo (con probabilidad de $2^{-100}$). Repetimos este proceso hasta que encontremos un primo.


In [7]:
def generar_primo(L):
    while True:
        possible_p = random.randint(10**(L-1), 10**L)
        if miller_rabin(possible_p, 100):
            return possible_p

### alg_ext_euclides es una función que busca el máximo común divisor entre 2 números a y b

alg_ext_euclides(a: int, b: int) -> (int, int, int)

Se implementa el conocido algoritmo de euclides. En particular esta vez entregamos los numeros s y t que tienen propiedades que seran de útilidad para otras funciones. s y t cumplen la propiedad de $gcd = s*a + n*t$ mod n

In [9]:
def alg_ext_euclides(a, b):
    if a == 0:
        return b, 0, 1

    new_a = b % a
    new_b = a

    gcd, x, y = alg_ext_euclides(new_a, new_b)
    s = y - (b // a) * x
    t = x

    return gcd, s, t

### inverso es una funcion que retorna el inverso modular de un numero a
inverso(a: int, n: int) -> int

Aprovechamos la propiedad que tienen el s y t que retorna alg_ext_euclides para encontrar el inverso. si el numero a y el modulo n son comprimos entonces existe un inverso. Ese inverso es simplement el número s % n.

como MCD = s*a + n*t

podemos extraer el n*t ya que en modulo n, n*t siempre va a ser 0.

MCD = s*a

como MCD es 1:

1 = s*a mod n

In [ ]:
def inverso(a, n):
    g, s, _ = alg_ext_euclides(a, n)
    if g != 1:
        return -1
    return s % n

### generar_clave es una funcion que genera una clave privada y una clave pública y las escribe en 2 archivos txt.

generar_clave(L:int) -> None

Utilizamos el protocolo RSA: buscamos 2 primos p y q. elegimos un e que sea coprimo con phi (7 es siempre coprimo porque es un primo). Definimos la clave publica como (e, N) y la privada como (d, N) donde d es el inverso de e en modulo phi.

In [11]:
def generar_clave(L):
    p = generar_primo(L//2)
    q = generar_primo(L//2)
    while alg_ext_euclides(p, q)[0] != 1:
        q = generar_primo(L//2)
    N = p*q
    phi = (p - 1)*(q - 1)
    e = 7
    d = inverso(e, phi)
    public_key = open(os.path.join(sys.path[0], "./public_key.txt"), "w")
    private_key = open(os.path.join(sys.path[0], "./private_key.txt"), "w")
    public_key.write(str(e) + "\n" + str(N))
    private_key.write(str(d) + "\n" + str(N))

### Las funciones enc y dec encriptan o decriptan un mensaje con las llaves definidas

enc(m: int) -> int

dec(c: int) -> int

enc toma el mensaje m y hace exponenciacion modular con la clave publica e.

dec toma un texto cifrado c y hace la exponenciacion modular con su clave secreta d

Ya que c = $m^e$ mod N y d es inverso de e, podemos hacer m = $m^{ed}$ mod N

In [13]:
def enc(m):
    key_file = open(os.path.join(sys.path[0], "./public_key.txt"), "r")
    e, N = [int(i.strip()) for i in key_file.readlines()]
    return exp_mod(m, e, N)

In [14]:
def dec(c):
    key_file = open(os.path.join(sys.path[0], "./private_key.txt"), "r")
    d, N = [int(i.strip()) for i in key_file.readlines()]
    return exp_mod(c, d, N)